## 0. Import Modules and Data

In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data
import pandas_datareader.data as web
import datetime as dt

In [2]:
import Portfolio as port # This is from the Portfolio.py file with the different classes defined

## 1. Set-up Portfolio Strategy

In [3]:
# Define start date and end date

start_date = '2015-01-01'

end_date = '2020-01-05'

In [4]:
# Define Shares

# Provide the name of the ticker and type (Equity or Bond)

voo = port.Share('VOO', 'Equity', start_date,end_date)
bnd = port.Share('BND', 'Bond', start_date, end_date)

In [5]:
voo.get_value('2019-01-01')

229.99

In [6]:
voo.get_value('2019-10-31')

278.55

In [7]:
# Need a way to group the different shares together

shares_list = [voo, bnd]

shares_dict = {}

for share in shares_list:
    shares_dict[share] = share.type


In [8]:
shares_dict

{<Portfolio.Share at 0x7f3cb40c0b00>: 'Equity',
 <Portfolio.Share at 0x7f3cb40c0d30>: 'Bond'}

In [9]:
# Define Strategy

# Provide the equity distribution, the bond distriubtion, cash distribution, and the threshold
strat = port.Strategy(50,50,0,5)

In [10]:
# Run portfolio with set start and end dates

portfolio = port.Portfolio(shares_dict)

portfolio.initial_buy(500, strat, start_date)

In [11]:
portfolio.get_asset_values(start_date)

In [12]:
portfolio.asset_split

{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}

In [13]:
# Run the portfolio over a series of months

time_period = pd.date_range(pd.to_datetime(start_date),pd.to_datetime(end_date))

for day in time_period:
    print(day)
    portfolio.reinvest_divs(day)
    portfolio.get_asset_values(day)
    print(portfolio.asset_split)
    if portfolio.asset_split['Equities'] > strat.equity_distribution+strat.threshold:
        sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*((portfolio.asset_split['Equities']-strat.equity_distribution)/100)
        sell_amt_per = sell_amt/len(portfolio.equities)
        for share in portfolio.equities: # sell equities and buy more bonds
            portfolio.sell(share, sell_amt_per, day)
        for share in portfolio.bonds:
            portfolio.buy(share, sell_amt_per, day)
               
    if portfolio.asset_split['Bonds'] > strat.bond_distribution+strat.threshold:
        sell_amt = (portfolio.asset_values['Equities']+portfolio.asset_values['Bonds'])*(portfolio.asset_split['Bonds']-strat.bond_distribution)
        sell_amt_per = sell_amt/len(portfolio.bonds)
        for share in portfolio.bonds: # sell bonds and buy more equities
            portfolio.sell(share, sell_amt_per, day)
        for share in portfolio.bonds:
            portfolio.buy(share, sell_amt_per, day)
        
    

2015-01-01 00:00:00
{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}
2015-01-02 00:00:00
{'Equities': 50.0, 'Bonds': 50.0, 'Cash': 0.0}
2015-01-03 00:00:00
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-04 00:00:00
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-05 00:00:00
{'Equities': 49.48439897186935, 'Bonds': 50.51560102813064, 'Cash': 0.0}
2015-01-06 00:00:00
{'Equities': 49.16513484755205, 'Bonds': 50.83486515244794, 'Cash': 0.0}
2015-01-07 00:00:00
{'Equities': 49.460491131767945, 'Bonds': 50.53950886823204, 'Cash': 0.0}
2015-01-08 00:00:00
{'Equities': 49.934970195913024, 'Bonds': 50.06502980408698, 'Cash': 0.0}
2015-01-09 00:00:00
{'Equities': 49.68145734582606, 'Bonds': 50.31854265417394, 'Cash': 0.0}
2015-01-10 00:00:00
{'Equities': 49.462145782702805, 'Bonds': 50.537854217297195, 'Cash': 0.0}
2015-01-11 00:00:00
{'Equities': 49.462145782702805, 'Bonds': 50.537854217297195, 'Cash': 0.0}
2015-01-12 00:00:00
{'E

{'Equities': 50.072033082259026, 'Bonds': 49.92796691774098, 'Cash': 0.0}
2015-04-05 00:00:00
{'Equities': 50.072033082259026, 'Bonds': 49.92796691774098, 'Cash': 0.0}
2015-04-06 00:00:00
{'Equities': 50.072033082259026, 'Bonds': 49.92796691774098, 'Cash': 0.0}
2015-04-07 00:00:00
{'Equities': 49.990036244892686, 'Bonds': 50.009963755107314, 'Cash': 0.0}
2015-04-08 00:00:00
{'Equities': 50.09864164377203, 'Bonds': 49.90135835622797, 'Cash': 0.0}
2015-04-09 00:00:00
{'Equities': 50.26121193849973, 'Bonds': 49.73878806150027, 'Cash': 0.0}
2015-04-10 00:00:00
{'Equities': 50.40992411006955, 'Bonds': 49.590075889930446, 'Cash': 0.0}
2015-04-11 00:00:00
{'Equities': 50.29070448639372, 'Bonds': 49.70929551360628, 'Cash': 0.0}
2015-04-12 00:00:00
{'Equities': 50.29070448639372, 'Bonds': 49.70929551360628, 'Cash': 0.0}
2015-04-13 00:00:00
{'Equities': 50.29070448639372, 'Bonds': 49.70929551360628, 'Cash': 0.0}
2015-04-14 00:00:00
{'Equities': 50.29131510321919, 'Bonds': 49.7086848967808, 'Cash

{'Equities': 51.353566121262695, 'Bonds': 48.646433878737305, 'Cash': 0.0}
2015-07-18 00:00:00
{'Equities': 51.39894964368784, 'Bonds': 48.60105035631217, 'Cash': 0.0}
2015-07-19 00:00:00
{'Equities': 51.39894964368784, 'Bonds': 48.60105035631217, 'Cash': 0.0}
2015-07-20 00:00:00
{'Equities': 51.39894964368784, 'Bonds': 48.60105035631217, 'Cash': 0.0}
2015-07-21 00:00:00
{'Equities': 51.25392857190615, 'Bonds': 48.74607142809385, 'Cash': 0.0}
2015-07-22 00:00:00
{'Equities': 51.195763707693565, 'Bonds': 48.80423629230643, 'Cash': 0.0}
2015-07-23 00:00:00
{'Equities': 50.99390264409875, 'Bonds': 49.006097355901254, 'Cash': 0.0}
2015-07-24 00:00:00
{'Equities': 50.716486899213656, 'Bonds': 49.283513100786344, 'Cash': 0.0}
2015-07-25 00:00:00
{'Equities': 50.52550650600007, 'Bonds': 49.47449349399993, 'Cash': 0.0}
2015-07-26 00:00:00
{'Equities': 50.52550650600007, 'Bonds': 49.47449349399993, 'Cash': 0.0}
2015-07-27 00:00:00
{'Equities': 50.52550650600007, 'Bonds': 49.47449349399993, 'Cas

{'Equities': 50.77939144467643, 'Bonds': 49.22060855532357, 'Cash': 0.0}
2015-10-29 00:00:00
{'Equities': 50.858914481696374, 'Bonds': 49.141085518303626, 'Cash': 0.0}
2015-10-30 00:00:00
{'Equities': 50.67694431788456, 'Bonds': 49.32305568211544, 'Cash': 0.0}
2015-10-31 00:00:00
{'Equities': 51.07264333989694, 'Bonds': 48.92735666010306, 'Cash': 0.0}
2015-11-01 00:00:00
{'Equities': 51.07264333989694, 'Bonds': 48.92735666010306, 'Cash': 0.0}
2015-11-02 00:00:00
{'Equities': 51.055613489160535, 'Bonds': 48.944386510839465, 'Cash': 0.0}
2015-11-03 00:00:00
{'Equities': 51.17751990837919, 'Bonds': 48.82248009162082, 'Cash': 0.0}
2015-11-04 00:00:00
{'Equities': 51.09101142250536, 'Bonds': 48.90898857749464, 'Cash': 0.0}
2015-11-05 00:00:00
{'Equities': 51.10600569878279, 'Bonds': 48.89399430121721, 'Cash': 0.0}
2015-11-06 00:00:00
{'Equities': 51.1942671883608, 'Bonds': 48.80573281163919, 'Cash': 0.0}
2015-11-07 00:00:00
{'Equities': 50.98121689718883, 'Bonds': 49.01878310281118, 'Cash':

{'Equities': 48.52944391239552, 'Bonds': 51.47055608760448, 'Cash': 0.0}
2016-02-03 00:00:00
{'Equities': 48.65679776634341, 'Bonds': 51.34320223365659, 'Cash': 0.0}
2016-02-04 00:00:00
{'Equities': 48.66978113515715, 'Bonds': 51.33021886484285, 'Cash': 0.0}
2016-02-05 00:00:00
{'Equities': 48.22042282276665, 'Bonds': 51.77957717723335, 'Cash': 0.0}
2016-02-06 00:00:00
{'Equities': 47.76805407302819, 'Bonds': 52.231945926971804, 'Cash': 0.0}
2016-02-07 00:00:00
{'Equities': 47.76805407302819, 'Bonds': 52.231945926971804, 'Cash': 0.0}
2016-02-08 00:00:00
{'Equities': 47.76805407302819, 'Bonds': 52.231945926971804, 'Cash': 0.0}
2016-02-09 00:00:00
{'Equities': 47.77843463584294, 'Bonds': 52.22156536415705, 'Cash': 0.0}
2016-02-10 00:00:00
{'Equities': 47.71064610938294, 'Bonds': 52.289353890617065, 'Cash': 0.0}
2016-02-11 00:00:00
{'Equities': 47.373375051108226, 'Bonds': 52.62662494889177, 'Cash': 0.0}
2016-02-12 00:00:00
{'Equities': 47.96189826898078, 'Bonds': 52.038101731019204, 'Cas

...

{'Equities': 46.53845102039142, 'Bonds': 53.46154897960859, 'Cash': 0.0}
2018-12-20 00:00:00
{'Equities': 46.1610296634082, 'Bonds': 53.8389703365918, 'Cash': 0.0}
2018-12-21 00:00:00
{'Equities': 45.66192603011551, 'Bonds': 54.33807396988449, 'Cash': 0.0}
2018-12-22 00:00:00
{'Equities': 45.02644862020282, 'Bonds': 54.97355137979718, 'Cash': 0.0}
2018-12-23 00:00:00
{'Equities': 45.02644862020282, 'Bonds': 54.97355137979718, 'Cash': 0.0}
2018-12-24 00:00:00
{'Equities': 45.010407894790305, 'Bonds': 54.98959210520968, 'Cash': 0.0}
2018-12-25 00:00:00
{'Equities': 46.288318989643614, 'Bonds': 53.71168101035641, 'Cash': 0.0}
2018-12-26 00:00:00
{'Equities': 46.288318989643614, 'Bonds': 53.71168101035641, 'Cash': 0.0}
2018-12-27 00:00:00
{'Equities': 46.46468826441666, 'Bonds': 53.53531173558335, 'Cash': 0.0}
2018-12-28 00:00:00
{'Equities': 46.37315735390409, 'Bonds': 53.626842646095916, 'Cash': 0.0}
2018-12-29 00:00:00
{'Equities': 46.533155794317175, 'Bonds': 53.466844205682804, 'Cash'

{'Equities': 48.76886308449859, 'Bonds': 51.231136915501416, 'Cash': 0.0}
2019-03-24 00:00:00
{'Equities': 48.76886308449859, 'Bonds': 51.231136915501416, 'Cash': 0.0}
2019-03-25 00:00:00
{'Equities': 48.76886308449859, 'Bonds': 51.231136915501416, 'Cash': 0.0}
2019-03-26 00:00:00
{'Equities': 48.94245553415827, 'Bonds': 51.05754446584173, 'Cash': 0.0}
2019-03-27 00:00:00
{'Equities': 48.77980101934804, 'Bonds': 51.22019898065194, 'Cash': 0.0}
2019-03-28 00:00:00
{'Equities': 48.864574460269274, 'Bonds': 51.135425539730726, 'Cash': 0.0}
2019-03-29 00:00:00
{'Equities': 49.04337843271562, 'Bonds': 50.95662156728438, 'Cash': 0.0}
2019-03-30 00:00:00
{'Equities': 49.49029558729381, 'Bonds': 50.5097044127062, 'Cash': 0.0}
2019-03-31 00:00:00
{'Equities': 49.49029558729381, 'Bonds': 50.5097044127062, 'Cash': 0.0}
2019-04-01 00:00:00
{'Equities': 49.474335472308454, 'Bonds': 50.52566452769154, 'Cash': 0.0}
2019-04-02 00:00:00
{'Equities': 49.47051023284922, 'Bonds': 50.52948976715078, 'Cash'

{'Equities': 49.71285893201755, 'Bonds': 50.28714106798245, 'Cash': 0.0}
2019-07-01 00:00:00
{'Equities': 49.697817159877836, 'Bonds': 50.302182840122164, 'Cash': 0.0}
2019-07-02 00:00:00
{'Equities': 49.72224892101696, 'Bonds': 50.27775107898304, 'Cash': 0.0}
2019-07-03 00:00:00
{'Equities': 49.86832299068173, 'Bonds': 50.13167700931825, 'Cash': 0.0}
2019-07-04 00:00:00
{'Equities': 49.95389660812948, 'Bonds': 50.04610339187051, 'Cash': 0.0}
2019-07-05 00:00:00
{'Equities': 49.95389660812948, 'Bonds': 50.04610339187051, 'Cash': 0.0}
2019-07-06 00:00:00
{'Equities': 49.831589800676596, 'Bonds': 50.16841019932341, 'Cash': 0.0}
2019-07-07 00:00:00
{'Equities': 49.831589800676596, 'Bonds': 50.16841019932341, 'Cash': 0.0}
2019-07-08 00:00:00
{'Equities': 49.831589800676596, 'Bonds': 50.16841019932341, 'Cash': 0.0}
2019-07-09 00:00:00
{'Equities': 49.89599377890996, 'Bonds': 50.10400622109004, 'Cash': 0.0}
2019-07-10 00:00:00
{'Equities': 49.990890219368715, 'Bonds': 50.00910978063129, 'Cas

{'Equities': 48.9982091486916, 'Bonds': 51.00179085130841, 'Cash': 0.0}
2019-10-08 00:00:00
{'Equities': 48.59542597579487, 'Bonds': 51.40457402420513, 'Cash': 0.0}
2019-10-09 00:00:00
{'Equities': 48.85668160606784, 'Bonds': 51.143318393932155, 'Cash': 0.0}
2019-10-10 00:00:00
{'Equities': 49.102991797530535, 'Bonds': 50.897008202469465, 'Cash': 0.0}
2019-10-11 00:00:00
{'Equities': 49.45440163597451, 'Bonds': 50.54559836402549, 'Cash': 0.0}
2019-10-12 00:00:00
{'Equities': 49.389845286839254, 'Bonds': 50.610154713160746, 'Cash': 0.0}
2019-10-13 00:00:00
{'Equities': 49.389845286839254, 'Bonds': 50.610154713160746, 'Cash': 0.0}
2019-10-14 00:00:00
{'Equities': 49.389845286839254, 'Bonds': 50.610154713160746, 'Cash': 0.0}
2019-10-15 00:00:00
{'Equities': 49.695874013402346, 'Bonds': 50.30412598659766, 'Cash': 0.0}
2019-10-16 00:00:00
{'Equities': 49.629917441587104, 'Bonds': 50.370082558412896, 'Cash': 0.0}
2019-10-17 00:00:00
{'Equities': 49.69732196365896, 'Bonds': 50.30267803634104,

OutOfBoundsDatetime: Out of bounds nanosecond timestamp 9223372800000000000

In [14]:
portfolio.cash_bal

0.0

In [15]:
portfolio.log

,Share,Action,Date,Amount


In [16]:
portfolio.val_hist

,Date,Total Value,Equities,Bonds


In [ ]:
sell_amt

In [ ]:
portfolio.shares

In [ ]:
portfolio.log

In [ ]:
portfolio.asset_split['Bonds']-strat.bond_distribution

In [ ]:
portfolio.asset_split['Bonds']

In [ ]:
246.38667/.502971399

In [ ]:
(246.38667+243.47552)*.002971399

In [ ]:
489.86218/2

In [ ]:
246.38667*.002971399

In [ ]:
output

In [ ]:
portfolio.log

In [ ]:
time_period

In [ ]:
start_date

In [ ]:
# Get the value a few days later

portfolio.get_value('2018-05-01')

In [ ]:
portfolio.get_asset_values('2018-05-01')

In [ ]:
portfolio.asset_values

In [ ]:
for share in portfolio.shares:
    print(share.div.index)

In [ ]:
portfolio.reinvest_divs('2018-05-01')

In [ ]:
# import sys
# !{sys.executable} -m pip install pandas_datareader

In [ ]:
## This is useful:
#https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html

In [ ]:
bnd = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp = web.DataReader('VOO', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
sp.head(3)

In [ ]:
bnd.head(3)

In [ ]:
bnd.loc['2017-07-07']['High']

In [ ]:
# Import dividend records
bnd_div = web.DataReader('BND', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')
sp_div = web.DataReader('VOO', 'yahoo-dividends', start='2015-10-30', end='2020-10-30')

In [ ]:
bnd_div.head(3)

In [ ]:
sp_div.head(3)

In [ ]:
sp_div.value.loc['2020-09-29']

In [ ]:
sp_div[sp_div.index < pd.to_datetime('2016-01-01')]

In [ ]:
sp_div.index

In [ ]:
str('sp')+str('_div')

In [ ]:
Assets = {}
Assets['Bonds'] = bnd
Assets['Equities'] = sp

In [ ]:
Dividends

In [ ]:
# Maybe it is worth merging the dividend and share price files together for easier reference?
# Or build a dictionary to allow easy mapping?

## 1. Calculate Value Given a Start and End Date

In [ ]:
start_date = '2015-11-20'
end_date = 

In [ ]:
def calc_value(share, divs, start, end, initial_value):
    # Take the average of the day's high and low for starting share price
    num_shares = initial_value/np.mean([share.loc[start]['Low'],share.loc[start]['High']])
    divs = divs[(divs.index <= pd.to_datetime(end)) & (divs.index >= pd.to_datetime(start))]
    for date in divs.index:
        price = np.mean([share.High.loc[date], share.Low.loc[date]])
        div_value = divs.value.loc[date]
        num_shares += div_value/price
    output = {}
    output['Value'] = num_shares*np.mean([share.loc[end]['Low'],share.loc[end]['High']])
    output['ROI'] = output['Value']-initial_value
    return(output)
    
    # Will need to factor in other actions later, but this will work for now
    
    


In [ ]:
calc_value(sp,sp_div, '2015-11-20', '2017-11-20',100)

In [ ]:
calc_value(bnd,bnd_div, '2015-11-20', '2017-11-20',100)

In [ ]:
sp.Close.plot()

In [ ]:
bnd.Close.plot()

In [ ]:
# Identify the ROI with different asset distributions

In [ ]:
class Share:

    def __init__(self, name, asset_type):
        """Initialize attributes."""
        self.name = name
        self.type = asset_type
        #self.amount = amount
        
    def get_value(self, date):
        df = web.DataReader(self.name, 'yahoo', start=date, end=date)
        return round(df.loc[date,'Close'],2)
    

In [ ]:
bnd = Share('BND','Bond')

In [ ]:
sp = Share('VOO','Equity')

In [ ]:
class Strategy:
    
    def __init__(self,equity_distribution, bond_distribution, cash_distribution, threshold):
        self.equity_distribution = equity_distribution
        self.bond_distribution = bond_distribution
        self.cash_distribution = 100-(equity_distribution+bond_distribution)
        self.threshold = threshold
    

In [ ]:
class Portfolio:
    
    def __init__(self):
        self.shares = {}
        self.log = {}
        self.cash_bal = 0
        self.asset_split = {'Equities': None,'Bonds': None,'Cash': None}
        self.asset_values = {'Equities': None,'Bonds': None,'Cash': None}
        
    def buy(self, share, amount, date):
        purchase_price = share.get_value(date)
        shares = amount/purchase_price
        # Record the transaction
        self.log[len(self.log)] = ['Buy',share,amount,purchase_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] += shares
        elif share not in self.shares:
            self.shares[share] = shares
            
    def sell(self, share, amount, date):
        sell_price = share.get_value(date)
        shares = amount/sell_price
        # Record the transaction
        self.log[len(self.log)] = ['Sell',share,amount,sell_price,shares,date]
        # Add the number of shares
        if share in self.shares:
            self.shares[share] -= shares
        elif share not in self.shares:
            self.shares[share] = shares
        cash_bal += amount
        
            
    def get_value(self,date):
        value = self.cash_bal
        # Iterate through the 'shares' attribute and get the current value for each 'Share' object
        for share in self.shares:
            value += share.get_value(date)*self.shares[share]
        return(value)
    
    def get_asset_values(self,date):
        bond_val = 0
        eq_val = 0
        for share in self.shares:
            if share.type == 'Bond':
                bond_val += share.get_value(date)*self.shares[share]
            elif share.type == 'Equity':
                eq_val += share.get_value(date)*self.shares[share]
        total = bond_val+eq_val+self.cash_bal
        self.asset_values = {'Equities': eq_val,'Bonds': bond_val,'Cash': self.cash_bal}
        self.asset_split = {'Equities': (eq_val/total)*100,'Bonds': (bond_val/total)*100,'Cash': (self.cash_bal/total)*100}
            
        
    

In [ ]:
portfolio = Portfolio()

In [ ]:
portfolio.get_value)

In [ ]:
# Calculate portfolio value

start_date = '2015-11-20'
end_date = '2020-11-20'

portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

portfolio.get_value(end_date)

In [ ]:
bond = web.DataReader('BND', 'yahoo', start='2015-10-30', end='2020-10-30')

In [ ]:
dates = list(bond.index)

In [ ]:
dates = [x.strftime('%Y-%m-%d') for x in dates]

In [ ]:
# Deploy balancing strategy

strategy = Strategy(75,25,0,1)

# Initial Purchase (manually allocated proportions)
portfolio = Portfolio() 
portfolio.buy(sp,750,start_date)
portfolio.buy(bnd,250,start_date)

for date in dates:
    portfolio.get_asset_values(date)
    if portfolio.asset_split['Equities'] > strategy.equity_distribution+strategy.threshold:
        amount_to_sell = (portfolio.asset_split['Equities']-strategy.equity_distribution)*portfolio.asset_values['Equities']
        portfolio.sell(sp,amount_to_sell,date)
        portfolio.buy(bnd,amount_to_sell,date)
        

    

In [ ]:
portfolio.get_asset_values('2019-10-29')

In [ ]:
portfolio.asset_split['Equities']